In [ ]:
import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)   
# pd.set_option('display.max_colwidth', None) 


import snowflake.connector
import os
import pandas as pd

# ENTER YOUR SNOWFLAKE CREDENTIALS
# If you don't have them here, or as environment variables, you'll be prompted on run.
SNOWFLAKE_USER = "" 
SNOWFLAKE_PASSWORD = ""

if SNOWFLAKE_USER == "":
    SNOWFLAKE_USER = os.environ.get("SNOWFLAKE_USER")
    if not SNOWFLAKE_USER or SNOWFLAKE_USER == "":
        SNOWFLAKE_USER = input("Enter your Snowflake username: ")
if SNOWFLAKE_PASSWORD == "":
    SNOWFLAKE_PASSWORD = os.environ.get("SNOWFLAKE_PASSWORD")
    if not SNOWFLAKE_PASSWORD or SNOWFLAKE_PASSWORD == "":
        SNOWFLAKE_PASSWORD = input("Enter your Snowflake password: ")

conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account='fougxvx-fb30115',
    warehouse='BOTANALYTICS_WH',
)

# Create a cursor object
cur = conn.cursor()



In [ ]:
## RUNNING THIS CELL WILL GET YOU THE GROUPINGS
from datetime import timedelta
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)   
pd.set_option('display.max_colwidth', None) 


cur.execute(f"""
            select BOT_ID, site_id,DATEADD(HOUR,-5,DATETIME) as datetime, storeason, hierarchy_component, event, subevent, actionecode, swversion, location_x, location_y,location_level, filename, velocity_linear, actions_0,transfertype
            from SYMBOTIC_KAFKA_DB.LOG_TABLES.DIAGNOSTICMESSAGE
           
            where site_id in ('walmart-tx-newbraunfels-site1') 
           
                AND event = 'collisionDetected'
                AND location_level != 0
                AND DATEADD(HOUR, -5, DATETIME) >= '2025-07-30 00:00:00' 
                AND DATEADD(HOUR, -5, DATETIME) <= '2025-07-31 00:00:00'
            order by datetime
        """)

rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]

df = pd.DataFrame(rows, columns=cols)



print(type(df['LOCATION_Y']))
df['LOCATION_Y'] = pd.to_numeric(df['LOCATION_Y'], errors='coerce')
df['LOCATION_X'] = pd.to_numeric(df['LOCATION_X'], errors='coerce')

## SPECIFIED THIS FOR TRANSFER DECK BUT COMMENT OUT IF YOU WANT ALL PLACES IN STRUCTURE
df = df[(df['LOCATION_Y'] >= 0) & (df['LOCATION_Y'] <= 8)]
df['DATETIME'] = pd.to_datetime(df['DATETIME'], errors='coerce')
df['DAY'] = df['DATETIME'].dt.strftime('%B %#d') 

df = df.sort_values(by='DATETIME')

df['GROUP_ID'] = np.nan  

def is_close(instance1, instance2):
    return (
        ## USED A ONE MINUTE INTEVAL BUT CAN CHANGE THE (seconds = ) IF YOU WANT DIFFERENT
        abs(instance1['DATETIME'] - instance2['DATETIME']) <= timedelta(seconds=60) and
        ## USED THIS FOR MAX X,Y DISTANCE BUT CHANGE THE NUMBER IF YOU WANT A DIF RANGE
        abs(instance1['LOCATION_X'] - instance2['LOCATION_X']) <= 2.5 and
        abs(instance1['LOCATION_Y'] - instance2['LOCATION_Y']) <= 2.5 and
        instance1['BOT_ID'] != instance2['BOT_ID']
    )

group_id = 1

for day in df['DAY'].unique():
    for level in df['LOCATION_LEVEL'].dropna().unique():
        dfFiltered = df[(df['DAY'] == day) & (df['LOCATION_LEVEL'] == level)].copy()
        dfFiltered.reset_index(inplace=True)

        seen = set()

        for idx, row in dfFiltered.iterrows():
            if idx in seen:
                continue

            group = [idx]
            queue = [idx]
            seen.add(idx)

            while queue:
                current_idx = queue.pop()
                current_row = dfFiltered.loc[current_idx]

                for other_idx, other_row in dfFiltered.iterrows():
                    if other_idx in seen:
                        continue
                    if is_close(current_row, other_row):
                        group.append(other_idx)
                        queue.append(other_idx)
                        seen.add(other_idx)
 
            if len(set(dfFiltered.loc[group, 'BOT_ID'])) > 1:
                df.loc[dfFiltered.loc[group, 'index'], 'GROUP_ID'] = group_id
                group_id += 1


result_df = df[df['GROUP_ID'].notna()].copy()
result_df = result_df.drop_duplicates(subset=['BOT_ID', 'GROUP_ID'])

result_df[['BOT_ID', 'GROUP_ID', 'DATETIME', 'LOCATION_X', 'LOCATION_Y', 'LOCATION_LEVEL', 'DAY']]
result_df.reset_index(drop=True)

result_df


In [ ]:
## RUNNING THIS CELL WILL TELL YOU WHAT GROUPING IDS HAVE AT LEAST ONE BOT WITH A STUCK ALARM BUT IT ISN'T NECESSARY

groupKeeper = set()
for _,rows1 in result_df.iterrows():
    if(rows1['GROUP_ID'] not in groupKeeper):
        bot = rows1['BOT_ID']
        start = rows1['DATETIME'] - pd.Timedelta(minutes=5)
        end = rows1['DATETIME'] + pd.Timedelta(minutes=5)
        cur.execute(f"""
            select BOT_ID, site_id,DATEADD(HOUR,-5,DATETIME) as datetime, storeason, hierarchy_component, event, subevent, actionecode, swversion, location_x, location_y,location_level, filename, velocity_linear, actions_0,transfertype

            from SYMBOTIC_KAFKA_DB.LOG_TABLES.DIAGNOSTICMESSAGE
            where site_id in ('walmart-tx-newbraunfels-site1') 
                And bot_id = '{bot}'

                AND event = 'alarm'
                AND location_level != 0
                AND DATEADD(HOUR, -5, DATETIME) >= '{start}'
                AND DATEADD(HOUR, -5, DATETIME) <= '{end}'
            order by datetime
        """)

        rows = cur.fetchall()
        cols = [desc[0] for desc in cur.description]

        newDf = pd.DataFrame(rows, columns=cols)
        
        val = False
        for _,newRows in newDf.iterrows():
            if(newRows['SUBEVENT'] == 'stuck' or newRows['SUBEVENT'] == 'estimation_error'):
                val = True
        
        if(val):
            groupKeeper.add(rows1['GROUP_ID'])
    

## RETURNS THE GROUP IDS WITH STUCK ALARMS IN AT LEAST ONE BOT
groupKeeper




